In [1]:
import requests
import sqlite3
import numpy as np
import pandas as pd
import datetime as dt
import os

con = sqlite3.connect('covid_cases.db',
                             detect_types=sqlite3.PARSE_DECLTYPES |
                             sqlite3.PARSE_COLNAMES)

In [162]:
sqlite3.__file__

'H:\\Program Files (x86)\\anaconda3\\envs\\EYML\\lib\\sqlite3\\__init__.py'

In [3]:
sqlite3.sqlite_version

'3.38.1'

# Download Covid Cases Data

In [2]:
# Use requests to get latest Covid data
%timeit

covid_data = requests.get('https://data.cdc.gov/api/views/9mfq-cb36/rows.csv')

In [3]:
covid_data_txt = [s for i,s in enumerate(covid_data.text.split('\n'))]

In [4]:
len(covid_data_txt)

47042

In [5]:
covid_data_txt[len(covid_data_txt)-1]

''

In [6]:
covid_data_txt[len(covid_data_txt)-2]

'05/27/2021,OH,1100312,921199,179113,732,196,19753,19753,0,0,0,05/28/2021 01:29:21 PM,Agree,Agree'

In [26]:
covid_data_txt

['submission_date,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,conf_death,prob_death,new_death,pnew_death,created_at,consent_cases,consent_deaths',
 '01/14/2022,KS,621273,470516,150757,19414,6964,7162,,,21,4,01/15/2022 02:59:30 PM,Agree,N/A',
 '01/02/2022,AS,11,,,0,0,0,,,0,0,01/03/2022 03:18:16 PM,,',
 '01/30/2022,CO,1240361,1133083,107278,0,0,11061,9754,1307,0,0,01/31/2022 04:55:36 PM,Agree,Agree',
 '07/09/2020,CO,36093,32964,3129,410,12,1706,1379,327,2,0,07/09/2020 12:00:00 AM,Agree,Agree',
 '03/08/2022,FL,5821791,,,1629,477,72209,,,30,3,03/08/2022 12:00:00 AM,Not agree,Not agree',
 '07/17/2020,MP,37,37,0,1,0,2,2,0,0,0,07/19/2020 12:00:00 AM,Agree,Agree',
 '08/11/2020,FL,537222,,,7898,160,11175,,,187,2,08/11/2020 12:00:00 AM,Not agree,Not agree',
 '01/01/2022,UT,636992,636992,0,0,0,3787,3635,152,0,0,01/03/2022 01:55:55 PM,Agree,Agree',
 '05/22/2021,MA,704796,659246,45550,451,46,17818,17458,360,5,0,05/23/2021 01:37:59 PM,Agree,Agree',
 '10/28/2020,PR,35112,34791,

In [21]:
covid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   submission_date  1000 non-null   object
 1   state            1000 non-null   object
 2   tot_cases        1000 non-null   object
 3   conf_cases       565 non-null    object
 4   prob_cases       565 non-null    object
 5   new_case         1000 non-null   object
 6   pnew_case        934 non-null    object
 7   tot_death        1000 non-null   object
 8   new_death        1000 non-null   object
 9   pnew_death       925 non-null    object
 10  created_at       1000 non-null   object
 11  consent_cases    887 non-null    object
 12  consent_deaths   887 non-null    object
 13  conf_death       465 non-null    object
 14  prob_death       465 non-null    object
dtypes: object(15)
memory usage: 117.3+ KB


In [5]:
(' text,').join(covid_data_txt[0].split(','))

'submission_date text,state text,tot_cases text,conf_cases text,prob_cases text,new_case text,pnew_case text,tot_death text,conf_death text,prob_death text,new_death text,pnew_death text,created_at text,consent_cases text,consent_deaths'

In [8]:
# Import SQL files

print("'''Create table covid_data ("+ (' text,').join(covid_data_txt[0].split(','))+")'''")

'''Create table covid_data (submission_date text,state text,tot_cases text,conf_cases text,prob_cases text,new_case text,pnew_case text,tot_death text,conf_death text,prob_death text,new_death text,pnew_death text,created_at text,consent_cases text,consent_deaths)'''


In [6]:
covid_data_text_list = []
for i in covid_data_txt[1:len(covid_data_txt)-1]:
    covid_data_text_list.append(tuple(i.split(',')))

In [20]:
covid_data_text_list[-1]

('05/27/2021',
 'OH',
 '1100312',
 '921199',
 '179113',
 '732',
 '196',
 '19753',
 '19753',
 '0',
 '0',
 '0',
 '05/28/2021 01:29:21 PM',
 'Agree',
 'Agree')

## Create covid_cases table in Current Database

In [16]:
'(' + ' text,'.join(covid_data_txt[0].split(',')) + ' text)'

'(submission_date text,state text,tot_cases text,conf_cases text,prob_cases text,new_case text,pnew_case text,tot_death text,conf_death text,prob_death text,new_death text,pnew_death text,created_at text,consent_cases text,consent_deaths text)'

In [12]:
dt.date.today().strftime('%b_%d')

'Mar_16'

In [14]:
db_name

'cdc_covid_cases_Mar_16'

In [23]:
query = "CREATE TABLE covid_data (" + (' text,'.join(covid_data_txt[0].split(',')))+ " text)"
query

'CREATE TABLE covid_data (submission_date text,state text,tot_cases text,conf_cases text,prob_cases text,new_case text,pnew_case text,tot_death text,conf_death text,prob_death text,new_death text,pnew_death text,created_at text,consent_cases text,consent_deaths text)'

In [7]:
# Create SQL tables for Covid, Census, State Abbreviations

cur = con.cursor()

todate = dt.date.today().strftime('%b_%d')
db_name = 'cdc_covid_cases_' + todate
dropdb_q = 'DROP TABLE IF EXISTS ' + db_name

cur.execute(dropdb_q)

# Create Covid Data Table
cases_query = "CREATE TABLE "+ db_name + " (" + (' text,'.join(covid_data_txt[0].split(',')))+ " text)"

cur.execute(cases_query)

# Insert Data into table
covid_data_text_list = []
for i in covid_data_txt[1:len(covid_data_txt)-1]:
    covid_data_text_list.append(tuple(i.split(',')))

    
insert_q = 'INSERT INTO {} VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)'.format(db_name)
cur.executemany(insert_q, covid_data_text_list)


con.commit()

#print(cur.fetchall())

#con.close()

In [8]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table'").fetchall()

[('state_abbreviations',), ('acs_5Y_2019',), ('cdc_covid_cases_Mar_16',)]

In [158]:
# con.interrupt()

In [9]:
covid_data_q = 'SELECT * FROM {}'.format(db_name)
con.execute(covid_data_q).fetchall()

[('01/14/2022',
  'KS',
  '621273',
  '470516',
  '150757',
  '19414',
  '6964',
  '7162',
  '',
  '',
  '21',
  '4',
  '01/15/2022 02:59:30 PM',
  'Agree',
  'N/A'),
 ('01/02/2022',
  'AS',
  '11',
  '',
  '',
  '0',
  '0',
  '0',
  '',
  '',
  '0',
  '0',
  '01/03/2022 03:18:16 PM',
  '',
  ''),
 ('01/30/2022',
  'CO',
  '1240361',
  '1133083',
  '107278',
  '0',
  '0',
  '11061',
  '9754',
  '1307',
  '0',
  '0',
  '01/31/2022 04:55:36 PM',
  'Agree',
  'Agree'),
 ('07/09/2020',
  'CO',
  '36093',
  '32964',
  '3129',
  '410',
  '12',
  '1706',
  '1379',
  '327',
  '2',
  '0',
  '07/09/2020 12:00:00 AM',
  'Agree',
  'Agree'),
 ('03/08/2022',
  'FL',
  '5821791',
  '',
  '',
  '1629',
  '477',
  '72209',
  '',
  '',
  '30',
  '3',
  '03/08/2022 12:00:00 AM',
  'Not agree',
  'Not agree'),
 ('07/17/2020',
  'MP',
  '37',
  '37',
  '0',
  '1',
  '0',
  '2',
  '2',
  '0',
  '0',
  '0',
  '07/19/2020 12:00:00 AM',
  'Agree',
  'Agree'),
 ('08/11/2020',
  'FL',
  '537222',
  '',
  '',
  

# State Abbreviations

In [10]:
# Load State Abbreviations from text file

state_abbrev = []
with open(os.path.join('..','datasets','state_abbrev.txt')) as f:
    for i,l in enumerate(f.readlines()):
        state_abbrev.append(','.join(l.strip('\n').split('|')))
        
f.close()

In [177]:
state_abbrev

['STATE,STUSAB,STATE_NAME,STATENS',
 '01,AL,Alabama,01779775',
 '02,AK,Alaska,01785533',
 '04,AZ,Arizona,01779777',
 '05,AR,Arkansas,00068085',
 '06,CA,California,01779778',
 '08,CO,Colorado,01779779',
 '09,CT,Connecticut,01779780',
 '10,DE,Delaware,01779781',
 '11,DC,District of Columbia,01702382',
 '12,FL,Florida,00294478',
 '13,GA,Georgia,01705317',
 '15,HI,Hawaii,01779782',
 '16,ID,Idaho,01779783',
 '17,IL,Illinois,01779784',
 '18,IN,Indiana,00448508',
 '19,IA,Iowa,01779785',
 '20,KS,Kansas,00481813',
 '21,KY,Kentucky,01779786',
 '22,LA,Louisiana,01629543',
 '23,ME,Maine,01779787',
 '24,MD,Maryland,01714934',
 '25,MA,Massachusetts,00606926',
 '26,MI,Michigan,01779789',
 '27,MN,Minnesota,00662849',
 '28,MS,Mississippi,01779790',
 '29,MO,Missouri,01779791',
 '30,MT,Montana,00767982',
 '31,NE,Nebraska,01779792',
 '32,NV,Nevada,01779793',
 '33,NH,New Hampshire,01779794',
 '34,NJ,New Jersey,01779795',
 '35,NM,New Mexico,00897535',
 '36,NY,New York,01779796',
 '37,NC,North Carolina,01027

In [13]:
# Create state_abbrev table in Current Database

cur.execute('DROP TABLE IF EXISTS state_abbreviations')

cur.execute('CREATE TABLE state_abbreviations (STATE INTEGER,STUSAB text,STATE_NAME text,STATENS INTEGER)')

state_abb_list = []
for i in state_abbrev:
    state_abb_list.append(tuple(i.split(',')))

con.executemany('INSERT INTO state_abbreviations VALUES (?,?,?,?)', state_abb_list[1:])

con.commit()

In [14]:
con.execute('SELECT * FROM state_abbreviations').fetchall()

[(1, 'AL', 'Alabama', 1779775),
 (2, 'AK', 'Alaska', 1785533),
 (4, 'AZ', 'Arizona', 1779777),
 (5, 'AR', 'Arkansas', 68085),
 (6, 'CA', 'California', 1779778),
 (8, 'CO', 'Colorado', 1779779),
 (9, 'CT', 'Connecticut', 1779780),
 (10, 'DE', 'Delaware', 1779781),
 (11, 'DC', 'District of Columbia', 1702382),
 (12, 'FL', 'Florida', 294478),
 (13, 'GA', 'Georgia', 1705317),
 (15, 'HI', 'Hawaii', 1779782),
 (16, 'ID', 'Idaho', 1779783),
 (17, 'IL', 'Illinois', 1779784),
 (18, 'IN', 'Indiana', 448508),
 (19, 'IA', 'Iowa', 1779785),
 (20, 'KS', 'Kansas', 481813),
 (21, 'KY', 'Kentucky', 1779786),
 (22, 'LA', 'Louisiana', 1629543),
 (23, 'ME', 'Maine', 1779787),
 (24, 'MD', 'Maryland', 1714934),
 (25, 'MA', 'Massachusetts', 606926),
 (26, 'MI', 'Michigan', 1779789),
 (27, 'MN', 'Minnesota', 662849),
 (28, 'MS', 'Mississippi', 1779790),
 (29, 'MO', 'Missouri', 1779791),
 (30, 'MT', 'Montana', 767982),
 (31, 'NE', 'Nebraska', 1779792),
 (32, 'NV', 'Nevada', 1779793),
 (33, 'NH', 'New Hampshire

In [92]:
cur.execute('SELECT name FROM sqlite_master WHERE type="table"').fetchall()

[('acs_5Y_2019',), ('cdc_covid_cases_Mar_16',), ('state_abbreviations',)]

# Census Data

In [59]:
# Load Census Data from local csv file
census = []
with open(os.path.join('..','datasets','ACSST5Y2019.S0101_2022-01-14T174326','ACSST5Y2019.S0101_data_with_overlays_2021-12-10T154120.csv')) as f:
    for i,l in enumerate(f.readlines()):
        census.append(l)
f.close()

census = census[1:]

In [60]:
# Clean data

census[0] = census[0].replace('!','_',).replace(' ','_')

In [61]:
len(census[0].split(','))

458

In [17]:
census[0]

'"id","Geographic_Area_Name","Estimate__Total__Total_population","Margin_of_Error__Total__Total_population","Estimate__Total__Total_population__AGE__Under_5_years","Margin_of_Error__Total__Total_population__AGE__Under_5_years","Estimate__Total__Total_population__AGE__5_to_9_years","Margin_of_Error__Total__Total_population__AGE__5_to_9_years","Estimate__Total__Total_population__AGE__10_to_14_years","Margin_of_Error__Total__Total_population__AGE__10_to_14_years","Estimate__Total__Total_population__AGE__15_to_19_years","Margin_of_Error__Total__Total_population__AGE__15_to_19_years","Estimate__Total__Total_population__AGE__20_to_24_years","Margin_of_Error__Total__Total_population__AGE__20_to_24_years","Estimate__Total__Total_population__AGE__25_to_29_years","Margin_of_Error__Total__Total_population__AGE__25_to_29_years","Estimate__Total__Total_population__AGE__30_to_34_years","Margin_of_Error__Total__Total_population__AGE__30_to_34_years","Estimate__Total__Total_population__AGE__35_to_39_y

In [79]:
# Create data tuples to load into SQL table.  Use only 16 columns

census_list = []
for i in census[1:]:
    temp = []
    for attr in i.split(','):
        temp.append(attr.strip('"'))
    census_list.append(tuple(temp)[:16])

In [80]:
census_list

[('0400000US01',
  'Alabama',
  '4876250',
  '*****',
  '293187',
  '535',
  '301064',
  '3234',
  '310456',
  '3238',
  '322486',
  '1334',
  '328180',
  '1250',
  '332622',
  '1103'),
 ('0400000US02',
  'Alaska',
  '737068',
  '*****',
  '53237',
  '239',
  '52810',
  '976',
  '49269',
  '986',
  '46704',
  '374',
  '54462',
  '453',
  '62205',
  '374'),
 ('0400000US04',
  'Arizona',
  '7050299',
  '*****',
  '433968',
  '304',
  '450570',
  '3446',
  '471732',
  '3465',
  '473352',
  '978',
  '488145',
  '908',
  '501361',
  '492'),
 ('0400000US05',
  'Arkansas',
  '2999370',
  '*****',
  '189636',
  '567',
  '196874',
  '2379',
  '197804',
  '2310',
  '201105',
  '1016',
  '202312',
  '1058',
  '202000',
  '981'),
 ('0400000US06',
  'California',
  '39283497',
  '*****',
  '2451528',
  '611',
  '2468226',
  '10131',
  '2575463',
  '10100',
  '2565170',
  '1451',
  '2751567',
  '1354',
  '3078492',
  '1016'),
 ('0400000US08',
  'Colorado',
  '5610349',
  '*****',
  '334032',
  '530'

In [82]:
a = ("INTEGER|"*16).split('|')

for i in [0,1,3]:
    a[i] = 'TEXT'    

In [84]:
a

['TEXT',
 'TEXT',
 'INTEGER',
 'TEXT',
 'INTEGER',
 'INTEGER',
 'INTEGER',
 'INTEGER',
 'INTEGER',
 'INTEGER',
 'INTEGER',
 'INTEGER',
 'INTEGER',
 'INTEGER',
 'INTEGER',
 'INTEGER',
 '']

In [85]:
census[0]

'"id","Geographic_Area_Name","Estimate__Total__Total_population","Margin_of_Error__Total__Total_population","Estimate__Total__Total_population__AGE__Under_5_years","Margin_of_Error__Total__Total_population__AGE__Under_5_years","Estimate__Total__Total_population__AGE__5_to_9_years","Margin_of_Error__Total__Total_population__AGE__5_to_9_years","Estimate__Total__Total_population__AGE__10_to_14_years","Margin_of_Error__Total__Total_population__AGE__10_to_14_years","Estimate__Total__Total_population__AGE__15_to_19_years","Margin_of_Error__Total__Total_population__AGE__15_to_19_years","Estimate__Total__Total_population__AGE__20_to_24_years","Margin_of_Error__Total__Total_population__AGE__20_to_24_years","Estimate__Total__Total_population__AGE__25_to_29_years","Margin_of_Error__Total__Total_population__AGE__25_to_29_years","Estimate__Total__Total_population__AGE__30_to_34_years","Margin_of_Error__Total__Total_population__AGE__30_to_34_years","Estimate__Total__Total_population__AGE__35_to_39_y

In [86]:
# Prepare Columns for census data table

census_col_name = []
for i in census[0].split(','):
    census_col_name.append(i)

In [87]:
census_col_name

['"id"',
 '"Geographic_Area_Name"',
 '"Estimate__Total__Total_population"',
 '"Margin_of_Error__Total__Total_population"',
 '"Estimate__Total__Total_population__AGE__Under_5_years"',
 '"Margin_of_Error__Total__Total_population__AGE__Under_5_years"',
 '"Estimate__Total__Total_population__AGE__5_to_9_years"',
 '"Margin_of_Error__Total__Total_population__AGE__5_to_9_years"',
 '"Estimate__Total__Total_population__AGE__10_to_14_years"',
 '"Margin_of_Error__Total__Total_population__AGE__10_to_14_years"',
 '"Estimate__Total__Total_population__AGE__15_to_19_years"',
 '"Margin_of_Error__Total__Total_population__AGE__15_to_19_years"',
 '"Estimate__Total__Total_population__AGE__20_to_24_years"',
 '"Margin_of_Error__Total__Total_population__AGE__20_to_24_years"',
 '"Estimate__Total__Total_population__AGE__25_to_29_years"',
 '"Margin_of_Error__Total__Total_population__AGE__25_to_29_years"',
 '"Estimate__Total__Total_population__AGE__30_to_34_years"',
 '"Margin_of_Error__Total__Total_population__AGE

In [88]:
# Create SQLite table

census_query = 'CREATE TABLE acs_5Y_2019 ('
for x,y in zip(census_col_name[:16], a):
    census_query += x + ' '+ y + ','
census_query = census_query[:-1] + ');'
census_query

cur.execute('DROP TABLE IF EXISTS acs_5Y_2019')
cur.execute(census_query)

In [25]:
census_query = 'CREATE TABLE acs_5Y_2019 ('
for x,y in zip(census_col_name[:16], a):
    census_query += x + ' '+ y + ','
census_query = census_query[:-1] + ');'
census_query

'CREATE TABLE acs_5Y_2019 ("id" TEXT,"Geographic_Area_Name" TEXT,"Estimate__Total__Total_population" TEXT,"Margin_of_Error__Total__Total_population" TEXT,"Estimate__Total__Total_population__AGE__Under_5_years" TEXT,"Margin_of_Error__Total__Total_population__AGE__Under_5_years" TEXT,"Estimate__Total__Total_population__AGE__5_to_9_years" TEXT,"Margin_of_Error__Total__Total_population__AGE__5_to_9_years" TEXT,"Estimate__Total__Total_population__AGE__10_to_14_years" TEXT,"Margin_of_Error__Total__Total_population__AGE__10_to_14_years" TEXT,"Estimate__Total__Total_population__AGE__15_to_19_years" TEXT,"Margin_of_Error__Total__Total_population__AGE__15_to_19_years" TEXT,"Estimate__Total__Total_population__AGE__20_to_24_years" TEXT,"Margin_of_Error__Total__Total_population__AGE__20_to_24_years" TEXT,"Estimate__Total__Total_population__AGE__25_to_29_years" TEXT,"Margin_of_Error__Total__Total_population__AGE__25_to_29_years" TEXT);'

In [89]:
# Load data into acs_5Y_2019 table

con.executemany('INSERT INTO acs_5Y_2019 VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)', census_list)

In [233]:
# Previous method.  DON'T USE

"""
con.execute('DROP TABLE IF EXISTS acs_5Y_2019')

# con.execute('''CREATE TABLE acs_5Y_2019 (? text, ? text)''', census_col_name[:2])
con.execute('''CREATE TABLE acs_5Y_2019 (id TEXT,
 'Geographic Area Name' TEXT,
 'Estimate__Total__Total population' INTEGER,
 'Margin of Error__Total__Total population' TEXT,
 'Estimate__Total__Total population__AGE__Under 5 years' INTEGER,
 'Margin of Error__Total__Total population__AGE__Under 5 years' INTEGER,
 'Estimate__Total__Total population__AGE__5 to 9 years' INTEGER,
 'Margin of Error__Total__Total population__AGE__5 to 9 years' INTEGER,
 'Estimate__Total__Total population__AGE__10 to 14 years' INTEGER,
 'Margin of Error__Total__Total population__AGE__10 to 14 years' INTEGER,
 'Estimate__Total__Total population__AGE__15 to 19 years' INTEGER,
 'Margin of Error__Total__Total population__AGE__15 to 19 years' INTEGER,
 'Estimate__Total__Total population__AGE__20 to 24 years' INTEGER,
 'Margin of Error__Total__Total population__AGE__20 to 24 years' INTEGER,
 'Estimate__Total__Total population__AGE__25 to 29 years' INTEGER,
 'Margin of Error__Total__Total population__AGE__25 to 29 years' INTEGER)''')
 
 """

In [90]:
cur.execute('SELECT * FROM acs_5Y_2019').fetchall()

[('0400000US01',
  'Alabama',
  4876250,
  '*****',
  293187,
  535,
  301064,
  3234,
  310456,
  3238,
  322486,
  1334,
  328180,
  1250,
  332622,
  1103),
 ('0400000US02',
  'Alaska',
  737068,
  '*****',
  53237,
  239,
  52810,
  976,
  49269,
  986,
  46704,
  374,
  54462,
  453,
  62205,
  374),
 ('0400000US04',
  'Arizona',
  7050299,
  '*****',
  433968,
  304,
  450570,
  3446,
  471732,
  3465,
  473352,
  978,
  488145,
  908,
  501361,
  492),
 ('0400000US05',
  'Arkansas',
  2999370,
  '*****',
  189636,
  567,
  196874,
  2379,
  197804,
  2310,
  201105,
  1016,
  202312,
  1058,
  202000,
  981),
 ('0400000US06',
  'California',
  39283497,
  '*****',
  2451528,
  611,
  2468226,
  10131,
  2575463,
  10100,
  2565170,
  1451,
  2751567,
  1354,
  3078492,
  1016),
 ('0400000US08',
  'Colorado',
  5610349,
  '*****',
  334032,
  530,
  353262,
  2858,
  360944,
  2870,
  358555,
  1224,
  377977,
  1156,
  439062,
  905),
 ('0400000US09',
  'Connecticut',
  3575074,

In [178]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table'").fetchall()

[('cdc_covid_cases_Mar_16',), ('state_abbreviations',), ('acs_5Y_2019',)]

# Join Datasets

Use `covid19_ETL.sql`, which works on Google Cloud Platform BigQuery, to combine datasets and generate a new table

In [165]:
with open('covid19_ETL.sql') as ETL:
    data_combine_query = ETL.read()
#print(data_combine_query)

ETL.close()

In [172]:
# Change BigQuery Database names to match SQLite Database names

data_combine_query = data_combine_query.replace(
                        '`coursera-analytics-class.covid_by_percent.cdc_covid_cases_Jan_19`', 
                        db_name)

data_combine_query = data_combine_query.replace(
                        '`coursera-analytics-class.covid_by_percent.',
                        '')

data_combine_query = data_combine_query.replace('`', '')


# Change Date Casting methods to work in SQLite
data_combine_query = data_combine_query.replace("CAST(REPLACE(submission_date, '/', '-') AS DATE FORMAT 'MM-DD-YYYY')",
                                                'DATE(SUBSTR(submission_date,7,4)||"-"||SUBSTR(submission_date,1,2)||"-"||SUBSTR(submission_date,4,2))')

data_combine_query = data_combine_query.replace(
                                        "CAST(REPLACE(cdc.submission_date, '/', '-') AS DATE FORMAT 'MM-DD-YYYY')",
                                        'DATE(SUBSTR(cdc.submission_date,7,4)||"-"||SUBSTR(cdc.submission_date,1,2)||"-"||SUBSTR(cdc.submission_date,4,2))')

# Change Float Casting methods to work in SQLite
data_combine_query = data_combine_query.replace("cs.new_cases/pop.Estimate__Total__Total_population",
                                                """CAST(cs.new_cases AS REAL)/(CAST(pop.Estimate__Total__Total_population AS REAL))""")

data_combine_query = data_combine_query.replace("rc.sum_cases_last_14_days/pop.Estimate__Total__Total_population",
                                                """CAST(rc.sum_cases_last_14_days AS REAL)/(CAST(pop.Estimate__Total__Total_population AS REAL))""")

data_combine_query = data_combine_query.replace('LEFT JOIN', 'INNER JOIN')

In [173]:
print(data_combine_query)

/* SQL query to combine CDC Covid 19 Cases, Census ACS 5-year Survey Results, 
and ANSI State Codes.  Calculations for 
1. new covid cases as percentage of population for individual states
2. sum of new covid cases on a 14-day basis, as a percentage of population for individual states    

Query was completed on Google Cloud Platform BigQuery.  The tables were named for files as follows:
1. datasets/United_States_COVID-19_Cases_and_Deaths_by_State_over_Time.csv AS cdc_covid_cases_Jan_13
2. datasets/ACSST5Y2019.S0101_2022-01-14T1743264/ACSST5Y2019.S0101_data_with_overlays_2021-12-10T154120.csv AS acs_5Y_2019 
3. datasets/state_abbrev.txt AS state_abbreviations
*/

WITH 
    data_cleaned AS(
        SELECT 
            DATE(SUBSTR(submission_date,7,4)||"-"||SUBSTR(submission_date,1,2)||"-"||SUBSTR(submission_date,4,2)) AS submit_date,
            CASE
                WHEN state='NYC' THEN 'NY'
                ELSE state
                END AS state_n,
            SUM(CAST(REPLACE(tot_cas

In [168]:
cur.execute(data_combine_query).fetchall()

[('2022-03-15',
  'AK',
  236001,
  0,
  1168,
  0,
  3426,
  737068,
  0.0,
  0.0046481464396772075),
 ('2022-03-15',
  'AL',
  1290171,
  438,
  18944,
  1,
  5610,
  4876250,
  8.98231222763394e-05,
  0.001150474237375032),
 ('2022-03-15',
  'AR',
  827336,
  423,
  10971,
  21,
  6659,
  2999370,
  0.00014102961621940608,
  0.0022201328945745274),
 ('2022-03-15',
  'AZ',
  1987318,
  0,
  28090,
  0,
  10428,
  7050299,
  0.0,
  0.0014790862061311158),
 ('2022-03-15',
  'CA',
  9031767,
  727,
  86792,
  -2,
  69310,
  39283497,
  1.8506499052261056e-05,
  0.0017643541256013944),
 ('2022-03-15',
  'CO',
  1327685,
  1294,
  11870,
  10,
  13007,
  5610349,
  0.00023064518802662723,
  0.0023183940963387483),
 ('2022-03-15',
  'CT',
  730811,
  305,
  10675,
  20,
  4069,
  3575074,
  8.531291939691319e-05,
  0.0011381582591017696),
 ('2022-03-15',
  'DC',
  135223,
  0,
  1326,
  0,
  545,
  692683,
  0.0,
  0.0007867956915356664),
 ('2022-03-15',
  'DE',
  258287,
  61,
  2811,
  4

In [169]:
# Use Pandas to execute query and write to a data frame

db_df = pd.read_sql_query(data_combine_query, con)

In [170]:
db_df

,submit_date,state_n,total_cases,new_cases,total_deaths,new_deaths,sum_cases_last_14_days,state_pop,new_cases_percent,cases_last_14_pop_percent
0,2022-03-15,AK,236001,0,1168,0,3426,737068,0.000000,0.004648
1,2022-03-15,AL,1290171,438,18944,1,5610,4876250,0.000090,0.001150
2,2022-03-15,AR,827336,423,10971,21,6659,2999370,0.000141,0.002220
3,2022-03-15,AZ,1987318,0,28090,0,10428,7050299,0.000000,0.001479
4,2022-03-15,CA,9031767,727,86792,-2,69310,39283497,0.000019,0.001764
...,...,...,...,...,...,...,...,...,...,...
40763,2020-01-22,VT,0,0,0,0,0,624313,0.000000,0.000000
40764,2020-01-22,WA,0,0,0,0,0,7404107,0.000000,0.000000
40765,2020-01-22,WI,0,0,0,0,0,5790716,0.000000,0.000000
40766,2020-01-22,WV,0,0,0,0,0,1817305,0.000000,0.000000


In [171]:
# Use Pandas to write df to csv

db_df.to_csv(os.path.join('..', 'datasets','Generated','US_'+dt.date.today().strftime('%b_%d')+'.csv'), index=False)

In [ ]:
con.close()